# Introduction to Linear Programming with PuLP

## Try me
 [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ffraile/operations-research-notebooks/blob/main/docs/source/CLP/libraries/Python%20PuLP%20Tutorial.ipynb)[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ffraile/operations-research-notebooks/main?labpath=docs%2Fsource%2FCLP%2Flibraries%2FPython%20PuLP%20Tutorial.ipynb)

## Requirements
### Install in your environment
#### Pip Installation
The simplest way to install PuLP in your environment is using [pip](https://pypi.org/project/pip/). If you have installed
Python and pip in your environment, just open a terminal and try:

```
pip install pulp
```
#### Conda Installation
If you use Conda, open a Conda Terminal and try: 

```
conda install –c conda-forge pulp
```

#### Google Colabs installation
Run the following code cell to try this notebook in Google Colabs:

In [ ]:
!pip install pulp


#### Binder installation
Run the following code cell to try this notebook in Binder:


In [ ]:
!pip install pulp
!pip install pandas
!pip install numpy


## Linear Optimisation with PulP
In this tutorial, we will learn to model and solve Linear Programming Problems using the Python open source Linear Programming library [PuLP](http://pythonhosted.org/PuLP/). PuLP can be installed using Conda, as described [here](https://anaconda.org/conda-forge/pulp).

To guide this example, we will use a simple LPP formulated in class:

maximise $z = 300x + 250y$

Subject to:

$2x + y \leq 40$  
$x + 3y \leq 45$  
$x \leq 12$  


In [3]:
# Let´s start importing the library PuLP to solve linear programs
import pulp
# We are going to use panda to display the results as tables using Panda
import pandas as pd
#And we will use numpy to perform array operations
import numpy as np
#We will use display and Markdown to format the output of code cells as Markdown
from IPython.display import display, Markdown

### Problem Class LpProblem
PuLP uses *classes* providing different methods to model and solve LPPs. The class that will contain our model is the **LpProblem** class. To create a new LpProblem we use the pulp LpProblem function:

- **LpProblem(name='None', sense=1):** Creates a new Linear Programming Problem. The parameter name (default 'None') assigns a name to the problem. The parameter sense (either pulp.LpMinimise or pulp.LpMaximize) sets the type of objective function. The default is minimize.

Let us create an instance for our problem:

In [4]:
# Create an instance of the problem class using LpProblem
model = pulp.LpProblem("Production_Mix_example", pulp.LpMaximize) #this will create an instance of an LP Maximise problem

### Variable class LpVariable
The definition of a LPP program with PuLP is very similar to the standard procedure used to model a problem. First, we need to define the unknown variables in our problem. For this purpose we use the class **LpVariable**. The function LpVariable allows us to create a variable:

- **LpVariable(name, lowBound=None, upBound=None, cat='Continuous', e=None):** Creates an instance of variable with the following properties:
    - **Name:** The name of the variable to be used in the solution. 
    - **lowBoud:** The lower bound of the variable, the default is unsrestricted (-Inf).
    - **upBound:** The upper bound of the variable. The default is unrestricted (Inf).
    - **cat:** Either 'Continuous' for continuous variables, 'Binary' for binary variables or 'Integer' for Integer variables. We will see in detail binary and integer variables in the course unit for Mixed Integer Programming, but now you know that you will be able to model and solve this type of problems with PuLP. The default is 'Continuous'.
    - **e:** This parameter is outside the scope of this course and can be neglected for now.

We can define the variables of our problem using the LpVariable function:

```python
x = pulp.LpVariable('x', lowBound=0, cat='Continuous')
y = pulp.LpVariable('y', lowBound=0, cat='Continuous')
```

Note however that using this function, we need a line of code for every unknown. This simply does not scale up. What if we have hundreds of unknowns? Luckily for us, PuLP provides a convenient method to write more efficient codes for our program, the **LpVariable.dicts** method, which basically allows us to create a set of variables with the same category, upper bounds and lower bounds at once:

- **LpVariable.dicts(name, indexs, lowBound=None, upBound=None, cat='Continuous')**: Creates a dictionary containing variables of type cat (default 'Continuous'), indexed with the keys contained in the *iterable* index and bounded by lowBound (default -Inf) and upBound (default Inf).

For instance, we can write the same code as:

In [5]:
# First we define a tuple with the variable names x and y
variable_names = ('x','y')
# Then we create a variable from a dictionary, using the variable names as keys
variables = pulp.LpVariable.dicts("vars",
                                     (i for i in variable_names),
                                     lowBound=0,
                                     cat='Continuous')

Notice that we have created a tuple with the variable names and then created a dictionary with the actual variables that we will use in our model. We will be able to get the variables from the dictionary using the names as keys. This way, if we had for instance 20 variables, we coud still create them only with two lines of code.

### Adding expressions
In PuLP, both objective function and constraints are *expressions* (algebraic expressions containing variables) that have to be added to the instance problem using the standard operand '+='. For instance, to add the objective function in this example, we could write:

```python
    model += 300 * x + 250 * y, "Profit"
```
With this line of code, we have added a new expression with name "Profit" that multiplies the technological coefficients to the variables X and Y (as defined in the code snippet in the previous section). This is the simplest way to create a expression, but it is clear that it is not the most scalable way, since we need to add a new term to the summation manually for every variable. PuLP provides a convenient function, *lpSum* to achieve this result programmatically. *lpSum* takes an array of expressions and returns the summation of the elements in the array. Let us see it action:

In [6]:
# We define the technological coefficients
coefficients = [300, 250]
# Then we add the objective function to the model like
# model += linear_expression, name
# eg model += 300*X + 250y, "Profit"
# We use the function lpSum to generate the linear expression from a vector
# The vector is generated using a for loop over the variable names:
model += (
    pulp.lpSum([
        coefficients[i] * variables[variable_names[i]]
        for i in range(len(variable_names))])
), "Profit"

Notice that we have used **list comprehension** to create the array passed to the lpSum function using an index array. In this case we have created an array of length equal to the number of variables using the functions **range** and **length**.
We can follow the same method to add constraints. For instance, the simplest way to add the constraint in our example is:
```python
# And the constraints
model += 2 * X + Y <= 40, "Man Power"
model += X + 3 * Y <= 45, "Machine Operating Time"
model += X <=12, "Marketing"
```
However, we can see that this is not the most scalable alternative there is, since we will need to add a new line of code every constraint. There is another approach, to put our data in iterable objects and use list comprehension and for loops to define the constraints:

In [7]:
# And the constraints, the Matrix A
A=[[2, 1], #Coefficients of the first constraint
   [1, 3], #Coefficients of the second constraint
   [1, 0]] #Coefficients of the third constraint

# And vector b
b = [40, 45, 12] #limits of the three constraints

# need We also define the name for the constraints
constraint_names = ['Man Power', 'Machine Operating Time', 'Marketing']
# Now we add the constraints using
# model += expression, name
# eg model += 2*X + y <= 40
# We add all constraints in a loop, using a vector and the function lpSum to generate the linear expression:
for i in range(len(A)):           
    model += pulp.lpSum([
        A[i][j] * variables[variable_names[j]] 
        for j in range(len(variable_names))]) <= b[i] , constraint_names[i]

#note that in this case all constraints are of type less or equal

Now that we have created our model, we can get the solution just by calling the method **solve()**. The status of the solution can be read in the LpStatus attribute:

In [8]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

'Optimal'

Now, let us display the solution in a nice table using Pandas. We are going to first display the solution value using markdown and then we will use Pandas to create a table with the results.

In [12]:
# Solution
max_z = pulp.value(model.objective)

#We use display and Mardown to show the value using markdown
display(Markdown("The value of the objective function is **%.2f**"%max_z))


# Print our decision variable values
display(Markdown("The following tables show the values obtained: "))
# First we create a dataframe from the dictionary of the solution. We want to use the variable indexes to present the results and 
# place the different values provided by the solver in the data frame.
var_df = pd.DataFrame.from_dict(variables, orient="index", 
                                columns = ["Variables"])
# First we add the solution. We apply a lambda function to get only two decimals:
var_df["Solution"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(float(item.varValue)))
# We do the same for the reduced cost:
var_df["Reduced cost"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(float(item.dj)))


# We use the display function to represent the results:
display(var_df)


# we define a dictionary with the constraints:
const_dict = dict(model.constraints)
#We create a list of records from the dictionary and exclude the Expression to have a more compact solution. 
con_df = pd.DataFrame.from_records(list(const_dict.items()), exclude=["Expression"], columns=["Constraint", "Expression"])

#Now we add columns for the solution, the slack and shadow price

con_df["Right Hand Side"] = con_df["Constraint"].apply(lambda item: "{:.2f}".format(-const_dict[item].constant))
con_df["Slack"] = con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].slack))
con_df["Shadow Price"] = con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].pi))

# And we display the results
display(con_df)


The value of the objective function is **6350.00**

The following tables show the values obtained: 

,Variables,Solution,Reduced cost
x,vars_x,12.00,0.00
y,vars_y,11.00,0.00


,Constraint,Right Hand Side,Slack,Shadow Price
0,Man_Power,40.00,5.00,-0.00
1,Machine_Operating_Time,45.00,-0.00,83.33
2,Marketing,12.00,-0.00,216.67


### Solved exercises
The following notebooks include exercises solved with PuLP, using different solvers: 

- [Making Chappie solved with CBC](../solved/Making%20Chappie%20(Solved%20CBC).ipynb)
- [Blending problem solved with CBC](../solved/Blending%20Problem%20(Solved%20CBC).ipynb)
- [Blending Craft Beer solved with CBC](../solved/Blending%20Craft%20Beer%20(Solved%20CBC).ipynb)
- [Blending Craft Beer solved with GRB](../solved/Blending%20Craft%20Beer%20(Solved%20GRB).ipynb)
- [Chappie II solved with CBC](../solved/Chappie%20II%20(Solved%20CBC).ipynb)
- [Manufacturing solar cell panels solved with CBC](../solved/Manufacturing%20solar%20cell%20panels%20(Solved%20CBC).ipynb)
- [Manufacturing solar cell panels solved with GRB](../solved/Manufacturing%20solar%20cell%20panels%20(Solved%20GRB).ipynb)
- [Petroleum Blending Solved with CBC](../solved/Petroleum%20Blending%20(Solved%20CBC).ipynb)